# Moist air properties

Let's consider a volume flow rate of 10 m³/s of humid air at 25 °C, relative humidity 80 % in an HVAC sytem located at 1000 m of altitude.

In these conditions, find:
- the partial pressure of the dry air;
- the water vapor conntent (or the humidity ration);
- the density of the humid air;
- the specific volume of the humid air;
- the specific enthalpy of the humid air;
- the mass flow rate of humid air;
- the mass flow rate of dry air;
- the mass flow rate of water vapor that needs to be added to the air flow in order to obtain saturated air.

In [1]:
V = 10   # m3/s, volume flow rate
θ = 25   # °C, air temperature
ϕ = 0.8  # -, relative humidity
z = 1000 # m, altitude

T = θ + 273.15 # K, air temperature

**Note**: [Unicode symbols](https://www.compart.com/en/unicode), such as the Greek ketters θ or ϕ, can be used. 

In [2]:
import numpy as np

# Physical constants
l = 2495e3  # J/kg, specific latent heat for vaprization
c = 1e3     # J/(kg K), specific heat of dry air
cv = 1.96e3 # J/(kg K), specific heat of water vapor

M = 28.9645 # kg/kmol, molar mass of dry air
Mv = 18.015 # kg/kmol, molar mass of water vapor
R = 8314    # J/(kmol K), gas constant

## Partial pressure of dry air

The partial pressure of the dry air, $p_{da}$, is the total pressure, $p$, minus the pressure of the of the vapors, $p_{v} = \varphi p_{vs}$:

$$ p_{da} = p - p_v = p - \varphi p_{vs}$$

In [3]:
p = 101.325e3 * (1 - 2.25577e-5 * z)**5.2559   # Pa, air pressure as a function of altitude

In [4]:
def pvs(θ: float) -> float:
    """
    Saturation vapor pressure in Pz as a function of tempetature θ in °C
    """
    import numpy as np
    T = θ + 273.15      # K, temperature
    # pws(T) saturation pressure over liquid water
    # for temp range [0 200] °C
    C8 = -5.8002206e3
    C9 = 1.3914993e0
    C10 = -4.8640239e-2
    C11 = 4.1764768e-5
    C12 = -1.4452093e-8
    C13 = 6.5459673e0
    return np.exp(C8/T + C9 + C10 * T + C11 * T **2 + C12 * T **3 + C13 * np.log(T))  # Pa

In [5]:
pv = ϕ * pvs(θ)    # Pa, vapor pressure as a function of relative humidity ϕ and temperature θ

In [6]:
pda = p - pv       # Pa, dry-air partial pressure
print("Partial pressure of the dry air is:", pda, "Pa")

Partial pressure of the dry air is: 87339.14623965876 Pa


In [7]:
print(f"Partial pressure of the dry air is: {pda/1000:.1f} kPa")

Partial pressure of the dry air is: 87.3 kPa


## Water content (humidity ratio)

In [8]:
w = Mv / M * pv / (p - pv)  # kg/kg, humidity ratio (mass vapor / mass dry air)
print(f"Water content is: {1000 * w:.2f} g/kg (mass of vapor per mass of dry air)")

Water content is: 18.06 g/kg (mass of vapor per mass of dry air)


## Density of moist air

The density of humid air is 
$$\rho = \frac{m_{ha}}{V} = \frac{M}{R} \frac{p}{T} - \varphi \frac{M - M_v}{R} \frac{p_{vs}}{T}$$

Density $\rho$ decreases with the increase in relative humidity, $\varphi$.

In [9]:
ρ = M / R * pda / T + Mv / R * pv / T   # kg/m^3, density

# To emphasize that the density of the moist air decreases with
# the increase of the  relative humidity:
ρ = M / R * p / T  - (M - Mv) / R * ϕ * pvs(θ) / T
print(f'Density of moist air is: {ρ:.2f} kg/m³')

Density of moist air is: 1.04 kg/m³


## Specific volume

In [10]:
v = R / Mv * (Mv / M + w) * T / p
print(f'Specific volume is: {v:.2f} m³/kg')

Specific volume is: 0.98 m³/kg


**Note**: The specific volume coresponds to the volume divided by the mass of dry air. Therefore $v \neq \frac{1}{\rho}$: the specific volume, $v = \frac{V}{m_{da}}$ is different of the massic volume  $\frac{1}{\rho} = \frac{V}{m_{ha}}$, where $m_{da}$ is the mass of dry air and $m_{ha}$ is the mass of hummid air. The mass of humid air is $m_{ha} = m_{da} + m_v$, where $m_{da}$ is the mass of dry-air and $m_v$ is the mass of vapor. 

In [11]:
print(f'Massic volume 1/ρ is: {1 / ρ:.2f} m³/kg')

Massic volume 1/ρ is: 0.96 m³/kg


## Specific enthalpy

In [12]:
h = c * θ + w * (l + cv * θ)
h1 = c * θ + w * l
print(f"Specific enthalpy is: {h/1000:.3f} kJ/kg")
print(f"Specific enthalpy with simplified formula is: {h1/1000:.3f} kJ/kg")
print(f"Relative error done by using the simplified formula: {(h1 - h) / h * 100:.3f} %")

Specific enthalpy is: 70.932 kJ/kg
Specific enthalpy with simplified formula is: 70.048 kJ/kg
Relative error done by using the simplified formula: -1.247 %


## Mass flow rate of humid air

In [13]:
mha = ρ * V     # kg/s mass flow rate of humid air
print(f'Mass flow rate of humid air is: {mha:.1f} kg/s')

Mass flow rate of humid air is: 10.4 kg/s


## Mass flow rate of dry air

In [14]:
mda = V / v     # kg/s, mass flow rate of humid air
print(f"Mass flow rate of dry air: {mda:.1f} kg/s")

Mass flow rate of dry air: 10.2 kg/s


## Mass flow rate of water for saturation

In [15]:
ws = Mv / M * pvs(θ) / (p - pvs(θ))    # kg/kg, water content at saturation
m = V / v                              # kg/s, mass flow rate of dry air
mv = (ws - w) * m                      # kg/s, mass flow of water needed to saturate
print(f'Mass flow rate of water needed to saturate the air flow is: \
{1000 * mv:.3f} g/s or {3600 * mv:.0f} kg/h.')

Mass flow rate of water needed to saturate the air flow is: 47.749 g/s or 172 kg/h.


## Bibliography

1. Fundamentals, ASHRAE Handbook (2009). Chapter 1: Psychrometrics. *American Society of Heating, Refrigerating and Air-Conditioning Engineers. Atlanta, USA*

2. TRANE (2012) [Psychrometry: One of the Fundamental Series](https://www.tranebelgium.com/files/book-doc/19/en/19.vwljh8zn.pdf)